# The Julia Programming Language

In [1]:
using BenchmarkTools

## Multiple Dispatch
Multiple dispatch is a fundamental paradigm of the Julia programming language.

## Lazy Evaluation
Lazy evaluation is the cornerstone of many functional programming languages. The key concept is as follows: when an operation such as a functional call is performed, we do not evaluate the result immediately (that would be _eager_ evaluation). Instead, the operation is postponed until the result is actually necessary, i.e., it is evaluated _lazily_.

Laziness is exploited in several packages in Julia: ``Lazy.jl``, ``LazyArrays.jl``, and many others (see https://www.juliapackages.com/packages?search=lazy). The finite element method package ``Gridap.jl`` also makes use of laziness to achieve very high performance.

### Laziness in Julia

### ``LazyArrays.jl``
Lazy arrays allow for the lazy evaluation of operations on arrays and matrices. This includes specific algebraic operations, matrix solve, kronecker product, broadcasting, etc.
The big advantage is that operations can be _compounded_ by not evaluating them immediately, but keeping track of which operations are to be performed on the array. 
- This allows for optimizations to be applied for certain compound operations that can be much faster than if every operation was evaluated eagerly.
- Additionally, memory usage can often be kept low when using lazy arrays: there is no need to allocate any memory for the result because it is not computed immediately. Only when the result is actually needed, it will be computed on-demand.

In [2]:
using LazyArrays

### Example 1: Optimized Matrix Operations

In [3]:
N = 5;
A = randn(N,N); x = randn(N); y = randn(N); z1 = similar(y); z2 = similar(y);

function fun(A, x, y, z)
    z .= @~ 2.0 * A * x + 3.0 * y
end
z1 = @btime fun(A, x, y, z1)       # Optimized
z2 = @btime 2.0 * A * x + 3.0 * y  # Not optimized

z1 ≈ z2      # But the result is the same

  94.953 ns (0 allocations: 0 bytes)
  206.055 ns (3 allocations: 288 bytes)


true

### Laziness in ``Gridap.jl``
Laziness is used extensively in ``Gridap.jl``, and is the one of the reasons for its excellent performance. Lazy arrays play a fundamental role in the implementation of the finite element method in Gridap (how it works is explained in https://gridap.github.io/Tutorials/dev/pages/t013_poisson_dev_fe/).
A large amount of code is devoted to lazy operations in Gridap, mostly centered around the ``lazy_map`` function. It creates a ``LazyArray`` that conceptually applies the mapping function to an array, but does not actually execute this until it is necessary. Large and complex 'compound' operations are built in this way. Eventually, the mapping may be evaluated at a set of points using evaluate.

Lazy objects constructed in this way provide the following advantages in Gridap:
 - Memory allocation (and consumption) is kept at very low levels, since ``lazy_map`` will never return an array that stores the result at all cells at once.
 - Computation can be done in an efficient way (e.g., using cache to store the entry-wise data without the need to allocate memory each time we access the ``LazyArray``).
 - The recursive application of ``lazy_map`` lets us build complex operation trees among arrays of ``Maps`` as the ones required for the implementation of variational forms. While building these trees, by virtue of Julia support for multiple type dispatching, there are plenty of opportunities for optimization by changing the order in which the operations are performed.
 - Using ``lazy_map`` we are hiding thousands of cell loops across the code. As a result, Gridap is much more expressive for cell-wise implementations.
 - ``lazy_map`` allows for very expressive code that reads much like the original mathematical formulation both internally (e.g., in pretty much every aspect of low-level Gridap code) and externally (e.g., the definition of the variational form, shown below).

In [ ]:
res(u, v) = ∫( ∇(u) ⋅ ∇(v) )dΩ - ∫( fsource * v  )dΩ;

Compare this to the original mathematical expression:
$$ \int \nabla u \cdot \nabla v d\Omega = \int f v d\Omega $$

The computational back-end of Gridap is discussed in the following paper: https://www.researchgate.net/publication/354890339_The_software_design_of_Gridap_a_Finite_Element_package_based_on_the_Julia_JIT_compiler

## Callable Structs

## Automatic Differentiation

## MapReduce